In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy
import os
import joblib
from sympy import symbols, diff
from matplotlib.widgets import Slider, Button
import matplotlib.ticker as mticker
import matplotlib
from ForceInfer import *

In [2]:
fname = '0.75Pa15p_cleared.csv'

In [3]:
tf.config.set_visible_devices([], 'GPU')   

We are going to use the file '0.75Pa15p_cleared.csv' as a demonstration of how to use the model. First let's specify some parameters in this data: 

In [4]:
val_begin = [.4] # Used for train-validation split. The percentage of validation data is always 10% and begins at this number. 
tau = 16 # how many frames used in the intergral of weak formation
nparticles = 15 # number of particles in this data
ndim = 4 # information for each particle, includes x, y, z and descriptor. If you want multi-dimension discriptors, you may go
# to the get_data function, and change n_dm 
n_out = 2 # the model only infers force in x_y plane, 2 dimensions. 
method = 'mean' # the method used to calculate the descriptor of each particles. You can also use 'std' for std(z_i) or a 
# callable, for example: method = np.mean
scale = 1 # the scale multiplied to the descriptor to make it more or less significant as an input to the model
deltat = 0.005 # the time interval between frames

# Define the symbol t for sympy expression
t = symbols('t')

# Define the function weight function w which is a sympy expression involving t
w = (t**2-1)**2

In [5]:
#the data should have 5 columns, each representing  

X1,X2,Y1,Y2 = get_data(fname,val_begin,w,tau,nparticles = nparticles,ndim = ndim,n_out = n_out,
                       method = method,scale = scale, deltat = deltat)

cannot find zmm


Now we get the input of train, input of validation, output of train, output of validation for this model. Note that X1 is a tuple of two tensors. The first tensor is 3-dimensional, representing $t$, $t'$ and each particles' information ($x$, $y$, $z$, $s$) respectively, used for inferring interaction and confinement force. The second tensor is 2-dimensional, $w$ convolve $v$, which multiplied by $\gamma$ is the damping The first dimension is $t$ and the second is $x$ or $y$. Y1 is 2-dimensional, $w$ convolve $a$.   

In [6]:
print(X1[0].shape)
print(X1[1].shape)
print(Y1.shape)

(132870, 15, 60)
(132870, 2)
(132870, 2)


Now we train the model. First we would want to set the parameter delta for huber loss to be 25% the std of training Y1 (we expect R^2 to be 0.99 so an average fitting error of 10%)

In [7]:
delta = Y1.numpy().std()* 0.25

In [9]:
mdl = MyModel1(w_to_coef(w, 16), nparticles, ndim, n_out, 32, 16)
h = decay_train_huber(mdl,X1,X2,Y1,Y2, nepochs = 200, delta = delta) # delta is the parameter for huber loss

Epoch 1/210
1039/1039 [==============================] - 12s 8ms/step - loss: 140951.6719 - r2: 0.9258 - M: 424945.1875 - val_loss: 57931.9492 - val_r2: 0.9745 - val_M: 133159.1719
Epoch 2/210
1039/1039 [==============================] - 6s 6ms/step - loss: 45243.7461 - r2: 0.9797 - M: 116062.5312 - val_loss: 98236.4531 - val_r2: 0.9576 - val_M: 221328.8125
Epoch 3/210
1039/1039 [==============================] - 6s 6ms/step - loss: 32557.3652 - r2: 0.9851 - M: 85564.5312 - val_loss: 28891.6406 - val_r2: 0.9879 - val_M: 63024.4180
Epoch 4/210
1039/1039 [==============================] - 6s 6ms/step - loss: 27143.8145 - r2: 0.9871 - M: 73664.2656 - val_loss: 31515.3516 - val_r2: 0.9867 - val_M: 69248.1328
Epoch 5/210
1039/1039 [==============================] - 6s 6ms/step - loss: 24174.4062 - r2: 0.9884 - M: 66749.9453 - val_loss: 24138.3086 - val_r2: 0.9898 - val_M: 53320.6367
Epoch 6/210
1039/1039 [==============================] - 6s 6ms/step - loss: 22189.6406 - r2: 0.9891 - M: 622

1039/1039 [==============================] - 6s 6ms/step - loss: 14784.2803 - r2: 0.9922 - M: 44546.1797 - val_loss: 17017.2188 - val_r2: 0.9915 - val_M: 44469.8555
Epoch 48/210
1039/1039 [==============================] - 6s 6ms/step - loss: 14722.2266 - r2: 0.9923 - M: 44211.7812 - val_loss: 13092.0820 - val_r2: 0.9930 - val_M: 36515.3789
Epoch 49/210
1039/1039 [==============================] - 6s 6ms/step - loss: 14715.8594 - r2: 0.9923 - M: 44200.4883 - val_loss: 13786.2246 - val_r2: 0.9929 - val_M: 37235.0625
Epoch 50/210
1039/1039 [==============================] - 6s 6ms/step - loss: 14679.4668 - r2: 0.9923 - M: 44186.2852 - val_loss: 14152.5752 - val_r2: 0.9928 - val_M: 37719.0156
Epoch 51/210
1039/1039 [==============================] - 6s 6ms/step - loss: 14623.7783 - r2: 0.9923 - M: 44050.6641 - val_loss: 14609.1924 - val_r2: 0.9926 - val_M: 38502.0820
Epoch 52/210
1039/1039 [==============================] - 6s 6ms/step - loss: 14589.1982 - r2: 0.9923 - M: 43865.9727 - val

1039/1039 [==============================] - 6s 6ms/step - loss: 13870.6631 - r2: 0.9927 - M: 41553.2773 - val_loss: 12917.6387 - val_r2: 0.9929 - val_M: 37137.9648
Epoch 94/210
1039/1039 [==============================] - 6s 6ms/step - loss: 13862.0703 - r2: 0.9928 - M: 41525.6289 - val_loss: 12506.6709 - val_r2: 0.9930 - val_M: 36760.3711
Epoch 95/210
1039/1039 [==============================] - 6s 6ms/step - loss: 13848.2402 - r2: 0.9928 - M: 41489.2266 - val_loss: 12153.3535 - val_r2: 0.9933 - val_M: 35015.4883
Epoch 96/210
1039/1039 [==============================] - 6s 6ms/step - loss: 13845.0674 - r2: 0.9928 - M: 41451.6406 - val_loss: 13333.1846 - val_r2: 0.9927 - val_M: 37910.5781
Epoch 97/210
1039/1039 [==============================] - 6s 6ms/step - loss: 13833.0762 - r2: 0.9928 - M: 41395.3281 - val_loss: 12635.8252 - val_r2: 0.9931 - val_M: 36217.5781
Epoch 98/210
1039/1039 [==============================] - 6s 6ms/step - loss: 13828.9766 - r2: 0.9928 - M: 41378.4258 - val

Epoch 139/210
1039/1039 [==============================] - 6s 6ms/step - loss: 13543.5508 - r2: 0.9929 - M: 40366.4180 - val_loss: 12459.1455 - val_r2: 0.9929 - val_M: 36984.9883
Epoch 140/210
1039/1039 [==============================] - 6s 6ms/step - loss: 13537.3125 - r2: 0.9930 - M: 40341.0273 - val_loss: 12211.8203 - val_r2: 0.9931 - val_M: 36221.0391
Epoch 141/210
1039/1039 [==============================] - 6s 6ms/step - loss: 13534.0146 - r2: 0.9930 - M: 40320.8828 - val_loss: 12227.7188 - val_r2: 0.9930 - val_M: 36595.4219
Epoch 142/210
1039/1039 [==============================] - 6s 6ms/step - loss: 13535.6807 - r2: 0.9930 - M: 40308.2227 - val_loss: 12495.7422 - val_r2: 0.9929 - val_M: 37212.0391
Epoch 143/210
1039/1039 [==============================] - 6s 6ms/step - loss: 13529.3555 - r2: 0.9930 - M: 40301.1680 - val_loss: 12398.8652 - val_r2: 0.9929 - val_M: 36924.4414
Epoch 144/210
1039/1039 [==============================] - 6s 6ms/step - loss: 13523.8096 - r2: 0.9930 - 

Epoch 185/210
1039/1039 [==============================] - 6s 6ms/step - loss: 13376.8145 - r2: 0.9931 - M: 39786.0977 - val_loss: 12022.3516 - val_r2: 0.9930 - val_M: 36354.1406
Epoch 186/210
1039/1039 [==============================] - 6s 6ms/step - loss: 13374.4678 - r2: 0.9930 - M: 39791.5898 - val_loss: 12074.7900 - val_r2: 0.9930 - val_M: 36446.7969
Epoch 187/210
1039/1039 [==============================] - 6s 6ms/step - loss: 13375.7568 - r2: 0.9931 - M: 39778.4102 - val_loss: 11983.7148 - val_r2: 0.9930 - val_M: 36295.8320
Epoch 188/210
1039/1039 [==============================] - 6s 6ms/step - loss: 13371.8242 - r2: 0.9930 - M: 39859.6992 - val_loss: 12064.4541 - val_r2: 0.9930 - val_M: 36343.5938
Epoch 189/210
1039/1039 [==============================] - 6s 6ms/step - loss: 13372.1299 - r2: 0.9931 - M: 39783.3828 - val_loss: 11979.8252 - val_r2: 0.9930 - val_M: 36288.1094
Epoch 190/210
1039/1039 [==============================] - 6s 6ms/step - loss: 13371.0312 - r2: 0.9931 - 

Let's get the damping coeffient for all particles

In [20]:
descriptors = X1[0][0,0,3::4]
print('particles descriptors', descriptors.numpy())
gamma = mdl.g_w(descriptors[...,tf.newaxis])[...,0].numpy()
print('gamma for all particles', gamma)

particles descriptors [-0.12295111  0.23337628  0.23149179  0.23492663 -0.05348454  0.222217
 -0.09414835 -0.15795815 -0.16096936 -0.06379275 -0.06908008  0.23170218
 -0.2676934  -0.0751785  -0.08845761]
gamma for all particles [0.6948838  0.9887138  0.98747504 0.98969525 0.7011998  0.98061395
 0.6994293  0.6817341  0.68006456 0.7009594  0.7008153  0.98761594
 0.5302875  0.700607   0.6998923 ]


Now let's predict environmental force. Note that the unit is 100mm/s^2

In [21]:
x = 1
y = 1
z = 0
s = -0.12
print('the confinement force has two element, x and y component, as described in the paper')
get_confinement(mdl,x,y,z,s)

the confinement force has two element, x and y component, as described in the paper


array([[-1.4558854, -1.8705156]], dtype=float32)

Interaction force

In [22]:
z1 = 0
z2 = 0
rho = 0.5
s1 = -0.12
s2 = 0.23
print('interaction force magnitude is')
get_interaction(mdl, z1, z2, rho,s1, s2)

interaction force magnitude is


array([1.55997932])